In [1]:
import sys
import os
import subprocess
import pytest
# Ensure 'src' is in sys.path to allow direct Python imports (e.g., for `find_chars`)
sys.path.insert(0, os.path.abspath(os.path.join("..", "..", "src")))

import myproject.cli

In [2]:
import logging
from io import StringIO

log_stream = StringIO()
handler = logging.StreamHandler(log_stream)
logger = logging.getLogger("myproject.cli")
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

print("Logger configured and handler added.")

Logger configured and handler added.


In [3]:
logger.info("Processing query...")

print("Logged 'Processing query...'")
print("Current stream contents:")
print(log_stream.getvalue())

Logged 'Processing query...'
Current stream contents:
Processing query...



In [4]:
log_stream.truncate(0)
log_stream.seek(0)

print("Log stream cleared.")
print("Current stream contents after clearing:")
print(log_stream.getvalue())

Log stream cleared.
Current stream contents after clearing:



In [5]:
logger.info("Another message")

print("Logged 'Another message'")
print("Stream contents:")
print(log_stream.getvalue())

Logged 'Another message'
Stream contents:
Another message



In [6]:
logger.removeHandler(handler)

logger.info("Message after handler removed")

print("Logged message after removing handler")
print("Stream contents (should be unchanged):")
print(log_stream.getvalue())

Logged message after removing handler
Stream contents (should be unchanged):
Another message



In [7]:
import logging
from io import StringIO
from unittest import mock

# Setup StringIO stream and logging handler
log_stream = StringIO()
handler = logging.StreamHandler(log_stream)
logger = logging.getLogger("myproject.cli")
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

# Mock sys.argv for CLI args
mock_args = ["myproject", "--query", "logtest", "--color=never"]

with mock.patch.object(sys, "argv", mock_args):
    try:
        myproject.cli.main()
    except SystemExit as e:
        exit_code = e.code

# Read captured logs
logs = log_stream.getvalue()

print("Exit code:", exit_code)
print("Captured logs:\n", logs)

# Cleanup handler after test
logger.removeHandler(handler)

[2025-05-29 15:01:30,484] [INFO] [DEV] Processing query...


[INFO] Processing query...


[2025-05-29 15:01:30,487] [DEBUG] [DEV] Simulating logic in DEV mode...


[DEBUG] Simulating logic in DEV mode...


[2025-05-29 15:01:30,490] [DEBUG] [DEV] DEV environment: Full diagnostics enabled


[RESULT]
Input query    : logtest
Processed (DEV MOCK): LOGTEST
[DEBUG] DEV environment: Full diagnostics enabled
Exit code: 0
Captured logs:
 


In [8]:
from myproject.core import process_query

# Define test cases: (input value, should_raise_exception)
test_cases = [
    (None, True),
    ("", True),
    ("   ", True),
    ("test", False),
    ("  hello  ", False),
]

for value, should_raise in test_cases:
    try:
        result = process_query(value)
        if should_raise:
            print(f"FAIL: Expected ValueError for input={repr(value)} but got {result}")
        else:
            expected = f"Processed query: {value.strip()}"
            if result == expected:
                print(f"PASS: input={repr(value)} → {result}")
            else:
                print(f"FAIL: input={repr(value)} → Expected '{expected}', got '{result}'")
    except ValueError as e:
        if should_raise:
            print(f"PASS: input={repr(value)} correctly raised ValueError")
        else:
            print(f"FAIL: Unexpected ValueError for input={repr(value)} → {e}")


PASS: input=None correctly raised ValueError
PASS: input='' correctly raised ValueError
PASS: input='   ' correctly raised ValueError
PASS: input='test' → Processed query: test
PASS: input='  hello  ' → Processed query: hello


In [9]:
import subprocess
import os
from pathlib import Path

# CLI invocation helper
def run_cli(args, env_vars=None):
    full_args = [sys.executable, "-m", "myproject"] + args + ["--color=never"]

    # Add src/ to PYTHONPATH for module discovery
    env = os.environ.copy()
    if env_vars:
        env.update(env_vars)
    env["PYTHONPATH"] = os.path.abspath("src")

    result = subprocess.run(
        full_args,
        capture_output=True,
        text=True,
        encoding="utf-8",
        env=env,  # ← Pass updated environment here
    )
    return result.stdout.strip(), result.stderr.strip(), result.returncode

log_path = Path("logs/myproject.log")
def read_log():
    return log_path.read_text(encoding="utf-8") if log_path.exists() else ""


In [10]:
os.environ["MYPROJECT_ENV"] = "DEV"

stdout, stderr, code = run_cli(["--query", "hello"])
print("STDOUT:\n", stdout)
print("STDERR:\n", stderr)
print("EXIT CODE:", code)


STDOUT:
 [INFO] Processing query...
[DEBUG] Simulating logic in DEV mode...
[RESULT]
Input query    : hello
Processed (DEV MOCK): HELLO
[DEBUG] DEV environment: Full diagnostics enabled
STDERR:
 [2025-05-29 15:01:30,684] [INFO] [DEV] Processing query...
[2025-05-29 15:01:30,684] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 15:01:30,684] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
EXIT CODE: 0


In [11]:
stdout, stderr, code = run_cli(["--query", " "])
print("STDOUT:\n", stdout)
print("STDERR:\n", stderr)
print("EXIT CODE:", code)


STDOUT:
 
STDERR:
 [2025-05-29 15:01:30,932] [ERROR] [DEV] [ERROR] argument -q/--query: Query string must not be empty
[ERROR] argument error: argument -q/--query: Query string must not be empty
usage: __main__.py [-h] -q QUERY [--version] [--quiet]
                   [--color {auto,always,never}]

MyProject CLI — a template command-line entry point.

options:
  -h, --help            show this help message and exit
  -q, --query QUERY     Search query or string to process (must not be empty)
  --version             Show version and exit
  --quiet               Suppress log messages
  --color {auto,always,never}
                        Colorized output: auto, always, or never
EXIT CODE: 1


In [12]:
print("Log file content:\n")
print(read_log())


Log file content:

[2025-05-29 14:58:49,188] [INFO] [DEV] Processing query...
[2025-05-29 14:58:49,191] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:58:49,194] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,536] [INFO] [DEV] Processing query...
[2025-05-29 14:59:04,537] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:59:04,539] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,773] [INFO] [DEV] Processing query...
[2025-05-29 14:59:04,773] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:59:04,773] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,943] [ERROR] [DEV] [ERROR] argument -q/--query: Query string must not be empty
[2025-05-29 14:59:05,160] [INFO] [UAT] Processing query...
[2025-05-29 14:59:05,160] [INFO] [UAT] UAT environment: Pre-production validation
[2025-05-29 15:00:34,154] [INFO] [DEV] Processing query...
[2025-05-29 15:00:34,158] [DEBUG] [DEV] Si

In [13]:
os.environ["MYPROJECT_ENV"] = "UAT"

stdout, stderr, code = run_cli(["--query", "uat test"])
print("STDOUT:\n", stdout)
print("STDERR:\n", stderr)
print("EXIT CODE:", code)

print("\nLog file content:\n")
print(read_log())


STDOUT:
 [INFO] Processing query...
[RESULT]
Input query    : uat test
Processed query: uat test
[INFO] UAT environment: Pre-production validation
STDERR:
 [2025-05-29 15:01:31,146] [INFO] [UAT] Processing query...
[2025-05-29 15:01:31,146] [INFO] [UAT] UAT environment: Pre-production validation
EXIT CODE: 0

Log file content:

[2025-05-29 14:58:49,188] [INFO] [DEV] Processing query...
[2025-05-29 14:58:49,191] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:58:49,194] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,536] [INFO] [DEV] Processing query...
[2025-05-29 14:59:04,537] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:59:04,539] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,773] [INFO] [DEV] Processing query...
[2025-05-29 14:59:04,773] [DEBUG] [DEV] Simulating logic in DEV mode...
[2025-05-29 14:59:04,773] [DEBUG] [DEV] DEV environment: Full diagnostics enabled
[2025-05-29 14:59:04,943] [ERRO

In [38]:
import myproject.cli

# Patch argv
sys.argv = ["myproject", "--query", "logtest", "--color=never"]

# Capture logs from the correct logger used in cli.py
log_stream = StringIO()
handler = logging.StreamHandler(log_stream)
logger = logging.getLogger("myproject")  # <- fix here
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)
logger.propagate = False

# Run CLI and capture exit
try:
    myproject.cli.main()
except SystemExit as e:
    exit_code = e.code

# Show results
logs = log_stream.getvalue()
print("#" * 40)
print("Captured Logs:\n", logs)
print("Exit Code:", exit_code)

# Clean up
logger.removeHandler(handler)

[2025-05-29 15:10:42,578] [INFO] [DEV] Processing query...


[INFO] Processing query...


[2025-05-29 15:10:42,580] [DEBUG] [DEV] Simulating logic in DEV mode...


[DEBUG] Simulating logic in DEV mode...


[2025-05-29 15:10:42,581] [DEBUG] [DEV] DEV environment: Full diagnostics enabled


[RESULT]
Input query    : logtest
Processed (DEV MOCK): LOGTEST
[DEBUG] DEV environment: Full diagnostics enabled
########################################
Captured Logs:
 Processing query...
Simulating logic in DEV mode...
DEV environment: Full diagnostics enabled

Exit Code: 0


In [1]:
import logging
import tempfile
from pathlib import Path
from myproject.cli_logger_utils import setup_logging
from myproject.constants import LOG_FILE_NAME
from myproject import settings

# Step 1: Create a temporary directory
temp_dir = tempfile.TemporaryDirectory()
log_path = Path(temp_dir.name) / LOG_FILE_NAME
print("Temporary log path:", log_path)

# Step 2: Override the LOG_DIR manually
settings.LOG_DIR = temp_dir.name  # CAUTION: this directly mutates the settings module!

# Step 3: Call setup_logging with reset=True and log_dir to ensure clean setup
setup_logging(log_dir=Path(settings.LOG_DIR), reset=True)

# Step 4: Get logger and write logs
logger = logging.getLogger("myproject")
logger.debug("Test debug log from notebook")
logger.info("Test info log from notebook")
logger.error("Test error log from notebook")

# Step 5: Check if log file exists
print("Log file exists?", log_path.exists())

# Step 6: Read and display log content
if log_path.exists():
    content = log_path.read_text(encoding="utf-8")
    print("Log content:\n", content)
else:
    print("Log file was not created.")


[2025-05-29 16:22:06,834] [DEBUG] [DEV] Logging initialized in C:\Users\HAMEDV~1\AppData\Local\Temp\tmp6bn5yvfh with level DEBUG
[2025-05-29 16:22:06,836] [DEBUG] [DEV] Test debug log from notebook
[2025-05-29 16:22:06,837] [INFO] [DEV] Test info log from notebook
[2025-05-29 16:22:06,838] [ERROR] [DEV] Test error log from notebook


Temporary log path: C:\Users\HAMEDV~1\AppData\Local\Temp\tmp6bn5yvfh\info.log
Log file exists? True
Log content:
 [2025-05-29 16:22:06,834] [DEBUG] [DEV] Logging initialized in C:\Users\HAMEDV~1\AppData\Local\Temp\tmp6bn5yvfh with level DEBUG
[2025-05-29 16:22:06,836] [DEBUG] [DEV] Test debug log from notebook
[2025-05-29 16:22:06,837] [INFO] [DEV] Test info log from notebook
[2025-05-29 16:22:06,838] [ERROR] [DEV] Test error log from notebook

